## This notebook contains the preprocessing we did to extract the data for serving it to the frontend

## Importing

In [ ]:
!pip install geemap

In [ ]:
!pip install PyCRS

In [ ]:
import geemap

In [ ]:
import ee

In [ ]:
import os

In [ ]:
import geopandas

In [ ]:
from geemap import geojson_to_ee, ee_to_geojson


2.   Authentication with Earth Engine is required to connect your project with Earth Engine



Run the ee.Authenticate function to authenticate your access to Earth Engine servers and ee.Initialize to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. More at: https://developers.google.com/earth-engine/guides/python_install

In case ee.Initialize() is not running, you need to create your own cloud EE project and enable API direclty to allow Colab to use EE features through this link: https://developers.google.com/earth-engine/cloud/earthengine_cloud_project_setup (follow the link on the section "Create a Cloud project" then proceed with the next section "Enable the Earth Engine API")

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize (project="master-mote-417820")

Alternatively: Open GEE Code Editor https://code.earthengine.google.com/, set up your own project manually (set up a non commerciable project) and enable permission for the GEE APIs for Colab.


```
ee.Initialize (project='your project name')
```





---



Now let's start with basic vizualization process, first you will be asked to upload the batch of shapefiles provided to you in your local file folder in collab. The Brazil.shp file serves as the official country boundary of our area of interest.

In [84]:
brazil_shapefile = geemap.shp_to_ee('/content/Brazil.shp')

## We calculate burned area per Land type

In [ ]:
land_cover_mapping = {
    0: "Water",
    1: "Evergreen Needleleaf Forest",
    2: "Evergreen Broadleaf Forest",
    3: "Deciduous Needleleaf Forest",
    4: "Deciduous Broadleaf Forest",
    5: "Mixed Forest",
    6: "Closed Shrublands",
    7: "Open Shrublands",
    8: "Woody Savannas",
    9: "Savannas",
    10: "Grasslands",
    11: "Permanent Wetlands",
    12: "Croplands",
    13: "Urban and Built-up",
    14: "Cropland/Natural Vegetation Mosaic",
    15: "Snow and Ice",
    16: "Barren or Sparsely Vegetated",
    254: "Unclassified"
}

In [ ]:
import ee
import geemap
import json

# Initialize the Earth Engine
ee.Initialize()

# Function to perform analysis for a given year
def analyze_year(year):
    # Load Land Cover for the given year
    landCover = ee.Image(f'MODIS/006/MCD12Q1/{year}_01_01').select('LC_Type1').clip(fc)

    totalLandCoverStats = landCover.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=fc.geometry(),
        scale=500,  # Scale in meters; adjust as needed for your analysis
        maxPixels=1e9
    ).getInfo()['LC_Type1']

    # Adjust the filter date range according to the year
    startDate = f'{year}-01-01'
    endDate = f'{year+1}-01-01'  # Assuming you want to cover the whole year

    # Define the dataset for burned areas and clip it to Brazil
    burnedArea = ee.ImageCollection('MODIS/061/MCD64A1') \
        .filter(ee.Filter.date(startDate, endDate)) \
        .select('BurnDate') \
        .map(lambda img: img.clip(fc)) \
        .reduce(ee.Reducer.sum())

    combinedBurnedArea = burnedArea

    # Mask the land cover image with the burned area data
    maskedLandCover = landCover.updateMask(combinedBurnedArea.gt(0))

    # Calculate the burned area for each land cover type
    burnedAreaByLandCover = maskedLandCover.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=fc.geometry(),
        scale=500,
        maxPixels=1e9
    )

    # Fetch the results
    burnedAreaStats = burnedAreaByLandCover.getInfo()['LC_Type1']

    # Convert pixel counts to hectares (assuming 25 ha per pixel)
    burnedAreaStats_hectares = {
    lc_type: {
        "burned_hectars": int(pixel_count * 25),
        "total_land_cover_hectares": int(totalLandCoverStats.get(str(lc_type), 0) * 25),
        "name": land_cover_mapping.get(int(lc_type), "Unknown Land Cover Type")
    } for lc_type, pixel_count in burnedAreaStats.items()
}
    return burnedAreaStats_hectares

# Initialize an empty dictionary to store results
yearly_results = {}

# Loop through each year and perform analysis
for year in range(2001, 2021):  # Adjust as needed
    yearly_results[year] = analyze_year(year)
    print(f"Processed year {year}")

# Convert the dictionary to JSON
json_output = json.dumps(yearly_results, indent=2)
print(json_output)

# Optionally, save the JSON to a file
with open('land_cover_burned_area_stats.json', 'w') as f:
    json.dump(yearly_results, f, indent=2)


Processed year 2001
Processed year 2002
Processed year 2003
Processed year 2004
Processed year 2005
Processed year 2006
Processed year 2007
Processed year 2008
Processed year 2009
Processed year 2010
Processed year 2011


## We calculate biomes land coverage

In [ ]:
biomes_shp_path = '/content/Brazil_biomes.shp'  # Adjust this path to your shapefile's location

# Convert the shapefile to an ee.FeatureCollection
biomes = geemap.shp_to_ee(biomes_shp_path)
biomes

In [ ]:
import ee
import geemap
import json

# Initialize the Earth Engine
ee.Initialize()

# Define the path to your biomes shapefile
biomes_shp_path = '/content/Brazil_biomes.shp'  # Adjust this path to your shapefile's location

# Convert the shapefile to an ee.FeatureCollection
biomes = geemap.shp_to_ee(biomes_shp_path)

# Function to calculate total burned area and total land cover area for a given year and biome
def analyze_year_for_biome(year, biome):
    # Load Land Cover for the given year and clip it to the current biome
    landCover = ee.Image(f'MODIS/006/MCD12Q1/{year}_01_01').select('LC_Type1').clip(biome.geometry())

    # Calculate the total area of the biome by counting the pixels and multiplying by the area per pixel (in hectares)
    pixelArea = ee.Image.pixelArea().divide(10000)  # Pixel area in hectares
    totalLandCoverArea = pixelArea.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=biome.geometry(),
        scale=500,  # Adjust scale as needed for your analysis
        maxPixels=1e9
    ).getInfo()['area']

    # Adjust the filter date range according to the year
    startDate = f'{year}-01-01'
    endDate = f'{year+1}-01-01'

    # Define the dataset for burned areas and clip it to the current biome
    burnedArea = ee.ImageCollection('MODIS/061/MCD64A1') \
        .filter(ee.Filter.date(startDate, endDate)) \
        .select('BurnDate') \
        .map(lambda img: img.clip(biome.geometry()))
    combinedBurnedArea = burnedArea.max()

    # Calculate the total burned area within the biome
    totalBurnedArea = pixelArea.updateMask(combinedBurnedArea.gt(0)).reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=biome.geometry(),
        scale=500,
        maxPixels=1e9
    ).getInfo()['area']

    return {
        "biome_id": biome.id().getInfo(),  # Adjust as needed for identifying biomes
        "total_burned_area_hectares": totalBurnedArea,
        "total_land_cover_area_hectares": totalLandCoverArea
    }

# Initialize an empty list to store results
yearly_results = []

# Loop through each year and each biome to perform analysis
for year in range(2010, 2021):  # Adjust years as needed
    for biome in biomes.getInfo()['features']:  # This might need optimization for large datasets
        biome_ee = ee.Feature(biome)  # Convert each biome back to an ee.Feature
        result = analyze_year_for_biome(year, biome_ee)
        yearly_results.append(result)
        print(f"Processed year {year} for biome {result['biome_id']}")

# Convert the list to JSON
json_output = json.dumps(yearly_results, indent=2)
print(json_output)

# Optionally, save the JSON to a file
with open('total_burned_land_cover_stats_by_biome.json', 'w') as f:
    json.dump(yearly_results, f, indent=2)



Processed year 2010 for biome 0
Processed year 2010 for biome 1
Processed year 2010 for biome 2
Processed year 2010 for biome 3
Processed year 2010 for biome 4
Processed year 2010 for biome 5
Processed year 2011 for biome 0
Processed year 2011 for biome 1
Processed year 2011 for biome 2
Processed year 2011 for biome 3
Processed year 2011 for biome 4
Processed year 2011 for biome 5
Processed year 2012 for biome 0
Processed year 2012 for biome 1
Processed year 2012 for biome 2
Processed year 2012 for biome 3
Processed year 2012 for biome 4
Processed year 2012 for biome 5
Processed year 2013 for biome 0
Processed year 2013 for biome 1
Processed year 2013 for biome 2
Processed year 2013 for biome 3
Processed year 2013 for biome 4
Processed year 2013 for biome 5
Processed year 2014 for biome 0
Processed year 2014 for biome 1
Processed year 2014 for biome 2
Processed year 2014 for biome 3
Processed year 2014 for biome 4
Processed year 2014 for biome 5
Processed year 2015 for biome 0
Processe

In [ ]:
map_dict = {
    "0": 'Caatinga',
    '1': "Cerrado",
    '2': "Pantanal",
    '3': "Pampa",
    '4': "Amazônia",
    '5': "Mata Atlântica"

}

print(map_dict)
for result in yearly_results:
    # The biome_id needs to be a string to match the keys in map_dict
    biome_id = result["biome_id"]
    # Check if the biome_id exists in map_dict to add the "name" field
    result["name"] = map_dict[biome_id]


{'0': 'Caatinga', '1': 'Cerrado', '2': 'Pantanal', '3': 'Pampa', '4': 'Amazônia', '5': 'Mata Atlântica'}


In [ ]:
yearly_results

In [ ]:
# Optionally, save the JSON to a file
with open('total_burned_land_cover_stats_by_biome.json', 'w') as f:
    json.dump(yearly_results, f, indent=2)

## We calcualte population densitiy in buried areas!

In [81]:
import ee
import geemap
import json

# Initialize the Earth Engine
ee.Initialize()

# A FeatureCollection defining Brazil boundary
fc = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_na', 'Brazil'))

# Load the WorldPop population dataset (mean over available years for simplification)
population_dataset = ee.ImageCollection("CIESIN/GPWv411/GPW_Population_Density").mean().clip(fc)

# Define the years for analysis
start_year = 2001
end_year = 2020  # Adjust the end year as needed

# Initialize a dictionary to hold the results
yearly_affected_population = {}

for year in range(start_year, end_year + 1):
    # Define the dataset for burned areas for the specific year and clip it to Brazil
    burned_area_dataset = ee.ImageCollection('MODIS/061/MCD64A1') \
        .filter(ee.Filter.date(f'{year}-01-01', f'{year+1}-01-01')) \
        .select('BurnDate') \
        .map(lambda img: img.clip(fc)).max()  # Use max() to create a single image representing all burned areas

    # Create a 10 km buffer around the burned areas
    # Note: In this modified script, we directly use the burned areas without applying a buffer
    burned_area_buffered = burned_area_dataset.gt(0)

    # Mask the WorldPop data with the burned areas
    masked_population_buffered = population_dataset.updateMask(burned_area_buffered)

    # Calculate the total population affected by the burned areas within Brazil
    total_affected_population_buffered = masked_population_buffered.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=fc.geometry(),  # Use the Brazil geometry
        scale=1000,  # Consider the resolution of the WorldPop data
        maxPixels=1e9
    )

    # Fetch the result
    total_affected_population_info_buffered = total_affected_population_buffered.getInfo()
    print(f"Total Affected Population in Brazil for {year}:", total_affected_population_info_buffered)

    # Add the result to the dictionary
    yearly_affected_population[year] = total_affected_population_info_buffered['population_density']

# Convert the dictionary to JSON
json_output = json.dumps(yearly_affected_population, indent=2)
print(json_output)

# Optionally, save the JSON to a file
with open('yearly_affected_population.json', 'w') as f:
    json.dump(yearly_affected_population, f, indent=2)


Total Affected Population in Brazil for 2001: {'population_density': 229418.72046944633}
Total Affected Population in Brazil for 2002: {'population_density': 608571.736472302}
Total Affected Population in Brazil for 2003: {'population_density': 493334.2900641658}
Total Affected Population in Brazil for 2004: {'population_density': 530647.8469181501}
Total Affected Population in Brazil for 2005: {'population_density': 536066.3685911056}
Total Affected Population in Brazil for 2006: {'population_density': 355553.6870754419}
Total Affected Population in Brazil for 2007: {'population_density': 780521.9477853382}
Total Affected Population in Brazil for 2008: {'population_density': 388134.3456334028}
Total Affected Population in Brazil for 2009: {'population_density': 268807.1870895398}
Total Affected Population in Brazil for 2010: {'population_density': 634968.0361256299}
Total Affected Population in Brazil for 2011: {'population_density': 395843.0577213366}
Total Affected Population in Bra

## We calculate and predict fires for the next year

In [ ]:
import geemap
import ee
import json

# Initialize the Google Earth Engine module
ee.Initialize()

# Load the JSON file with points
with open('/content/output.json', 'r') as file:
    points_list = json.load(file)

# Convert the list of points to ee.Feature objects
features = []
for point in points_list:
    # Ensure 'latitude' and 'longitude' keys exist
    if 'latitude' in point and 'longitude' in point:
        ee_point = ee.Geometry.Point([point['longitude'], point['latitude']])
        features.append(ee.Feature(ee_point))

# Create a FeatureCollection from the list of ee.Feature objects
points_feature_collection = ee.FeatureCollection(features)

# Create a map
Map = geemap.Map(center=[-23, -53], zoom=6)

# Add the FeatureCollection as a single layer
Map.addLayer(points_feature_collection, {}, 'Points')

# Optional layers (if defined elsewhere in your code)
# Map.addLayer(ba_clip, burnedAreaVis, 'Burned Area')
# Map.addLayer(brazil_shapefile, {'color': 'red'}, 'Brazil', opacity=0.5)

Map.addLayerControl()  # Add layer control to toggle layers

# Display the map
Map
